In [6]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8030/v1",
    api_key="token-abc123",
)

completion = client.chat.completions.create(
  model="/data/hf_cache/models/Qwen/Qwen2-0.5B-Instruct",
  messages=[
    {"role": "user", "content": """### 任务
请你使用中文回答，你是一位出色的信息抽取专家，目前需要对关于口腔鳞状细胞癌的病理诊断的中文医疗文本进行信息抽取，需要标注的文本在最后，下面是标注的要求：
### 要求
1. 我将会给你一段标注后的数据文本，是由实体和属性组成的一段数据，组成的格式为"实体:属性"你需要从这些数据中抽取关键信息填写json schema
2. 给你的实体与属性如下：分化信息，HPV，P16，切缘，神经侵犯，脉管侵犯，解剖学部位，解剖学部位:解剖学方位，解剖学部位:原发灶大小，解剖学部位:DOI，淋巴结清扫区域，淋巴结清扫区域:LN清扫方位，淋巴结清扫区域:LN数量，淋巴结清扫区域:颈清直径，淋巴结清扫区域:阳性LN数量，淋巴结清扫区域:胞膜外侵犯
3.若没有提及"阳性LN数量"其个数就是为0，由于阳性淋巴结数量为0，因此所有区域均不符合记录条件，最终"阳性淋巴结描述信息"数组为空
4.注意：淋巴结清扫区域:颈清直径中给的是一组范围的数据，通常以厘米为单位需要将厘米单位转换为毫米单位作为判断，例如"0.3-1.3cm",取其最大值"1.3cm"转换为毫米为单位为13mm作为判断来填写json schema，注意在填写这一项时："≤3"是指颈清直径小于等于3毫米，"≤6（＞3）"是指颈清直径大于3毫米小于等于6毫米，"＞6"是指颈清直径大于6毫米
5.若没有文本输入则全部判断为无阳性淋巴结描述信息"数组为空"
6.请你根据以上要求以及JSON schema对输入的文本进行信息抽取
JSON schema 如下：
{
  "$defs": {
    "DifferentiationLevel": {
      "enum": [
        "高分化",
        "高-中分化",
        "中分化",
        "中-低分化",
        "低分化",
        "无"
      ],
      "title": "DifferentiationLevel",
      "type": "string"
    },
    "HPVStatus": {
      "enum": [
        "阴性",
        "阳性",
        "无"
      ],
      "title": "HPVStatus",
      "type": "string"
    },
    "MarginStatus": {
      "enum": [
        "阴性",
        "阳性",
        "无"
      ],
      "title": "MarginStatus",
      "type": "string"
    },
    "P16Status": {
      "enum": [
        "阴性",
        "阳性",
        "无"
      ],
      "title": "P16Status",
      "type": "string"
    }
  },
  "properties": {
    "分化程度": {
      "$ref": "#/$defs/DifferentiationLevel"
    },
    "HPV检测": {
      "$ref": "#/$defs/HPVStatus"
    },
    "P16检测": {
      "$ref": "#/$defs/P16Status"
    },
    "切缘检测": {
      "$ref": "#/$defs/MarginStatus"
    },
    "神经侵犯": {
      "title": "神经侵犯",
      "type": "boolean"
    },
    "脉管侵犯": {
      "title": "脉管侵犯",
      "type": "boolean"
    }
  },
  "required": [
    "分化程度",
    "HPV检测",
    "P16检测",
    "切缘检测",
    "神经侵犯",
    "脉管侵犯"
  ],
  "title": "OtherDescription",
  "type": "object"
}
### 输入文本
切缘:送检切缘：“前、后、上、下、底”均阴性（-）
分化信息:高-中分化
P16:P16-
切缘:送检切缘：“前、后、上、下、底”均阴性（-）"""}
  ],
  temperature=0.0
)

print(completion.choices[0].message.content)

```json
{
  "分化程度": "高-中分化",
  "HPV检测": "阴性",
  "P16检测": "阴性",
  "切缘检测": "阴性",
  "神经侵犯": "无",
  "脉管侵犯": "无"
}
```


In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="sk-d7b518bdce9c43cba260186ff84cfc2a",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

completion = client.chat.completions.create(
    model="qwq-32b", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {
            'role': 'user', 
            'content': """"""}
        ],
    stream=True,
    temperature=0.0
)
reasoning_content = ""  # 定义完整思考过程
answer_content = ""     # 定义完整回复
is_answering = False   # 判断是否结束思考过程并开始回复

for chunk in completion:
    if not chunk.choices:
        print("\nUsage:")
        print(chunk.usage)
    else:
        delta = chunk.choices[0].delta
        # 打印思考过程
        if hasattr(delta, 'reasoning_content') and delta.reasoning_content != None:
            print(delta.reasoning_content, end='', flush=True)
            reasoning_content += delta.reasoning_content
        else:
            # 开始回复
            if delta.content != "" and is_answering is False:
                print("\n" + "=" * 20 + "完整回复" + "=" * 20 + "\n")
                is_answering = True
            # 打印回复过程
            print(delta.content, end='', flush=True)
            answer_content += delta.content


In [36]:
import json
import os
from typing import List, Dict, Tuple, Any, Optional
from collections import defaultdict
from tqdm import tqdm

LABEL_LIST_DICT = {
    "xbs":["既往口腔病灶切除史","既往口腔病灶切除史:既往口腔病灶切除时间","既往颈淋巴结清扫史","既往颈淋巴结清扫史:既往颈淋巴结清扫时间","既往化疗史","既往化疗史:既往化疗时间","既往化疗史:既往化疗方案","既往化疗史:既往化疗次数","既往放疗史","既往放疗史:既往放疗时间","既往放疗史:既往放疗方式","既往放疗史:既往放疗次数","肿瘤病灶性质","既往黏膜病史"],
    "blzd":["分化信息","HPV","P16","切缘","神经侵犯","脉管侵犯","解剖学部位","解剖学部位:解剖学方位","解剖学部位:原发灶大小","解剖学部位:DOI","淋巴结清扫区域","淋巴结清扫区域:LN清扫方位","淋巴结清扫区域:LN数量","淋巴结清扫区域:颈清直径","淋巴结清扫区域:阳性LN数量","淋巴结清扫区域:胞膜外侵犯"],
    "hys":["婚姻状况","生育状况"],
    "grs":["吸烟史","饮酒史","槟榔史"],
    "jws":["高血压","糖尿病","冠心病","血液系统疾病","其他肿瘤史","抗凝药物史"],
    "zkjc":["阳性淋巴结区域","阳性淋巴结区域:阳性淋巴结大小","阳性淋巴结区域:阳性淋巴结个数","阳性淋巴结区域:阳性淋巴结方位","阳性淋巴结区域:阳性淋巴结描述","阳性淋巴结区域:包膜外侵犯","病灶部位","病灶部位:病灶大小","病灶部位:病灶方位","病灶部位:累及部位"],
    "yxjc":["影像检查类型","阳性淋巴结区域","阳性淋巴结区域:阳性淋巴结大小","阳性淋巴结区域:阳性淋巴结个数","阳性淋巴结区域:阳性淋巴结方位","阳性淋巴结区域:阳性淋巴结描述","阳性淋巴结区域:包膜外侵犯","病灶部位","病灶部位:累及部位","病灶部位:病灶大小","病灶部位:病灶方位"],
    "ssjl":["手术名称","术后患者去向","主要病变","主要病变:病变部位","主要病变:病变大小","主要病变:病变影响对象","主要病变:病变影响结果","移除处理","移除处理:处理对象（删除）","移除处理:处理对象及所在部位","修复重建技术","修复重建技术:材料大小","修复重建技术:材料来源","修复重建技术:修复重建部位","修复重建技术:缺损大小","出血标志","出血标志:出血量"]
}
def group_labels(label_list_dict:Dict[str,List[str]],text_type:str) -> Dict[str, List[str]]:
    label_list = label_list_dict[text_type]
    grouped = {}
    attr_groups = {}
    non_attr_entities = set()

    for label in label_list:
        if ":" in label:
            main_entity, _ = label.split(":", 1)
            if main_entity not in attr_groups:
                attr_groups[main_entity] = [main_entity]
            attr_groups[main_entity].append(label)
        else:
            non_attr_entities.add(label)
            
    non_attr_entities -= set(attr_groups.keys())
    grouped["non_attr_entities"] = list(non_attr_entities)
    for key, values in attr_groups.items():
        grouped[key] = values
    return grouped

def match_grouped_label_data(label_list_dict, alpaca_data: List[Dict[str, Any]]) -> Dict[str, Any]:
    grouped_label_data = group_labels(label_list_dict, "ssjl")
    all_data_group_result = []

    for item in alpaca_data:
        group_result = {group_name: [] for group_name in grouped_label_data.keys()}
        labeled_origin_text = item.get("input", "")
        labeled_data_list = item.get("output", "").split("\n")

        for group_name, label_group in grouped_label_data.items():
            # print(group_name,label_group)
            if group_name != "non_attr_entities":
                i = 0
                while i < len(labeled_data_list):
                    j = i + 1
                    entity = labeled_data_list[i]
                    entity_len = len(entity.split(":"))
                    if group_name in entity:
                        while j < len(labeled_data_list) and labeled_data_list[j].startswith(entity.split(":")[0]):
                            if len(labeled_data_list[j].split(":")) == entity_len:
                                break
                            j = j + 1
                        group_result[group_name].append(labeled_data_list[i:j])
                        i = j 
                    else:
                        i += 1
            else:
                for entity in labeled_data_list:
                    entity_key = entity.split(":")[0]
                    if entity_key in label_group:
                        group_result[group_name].append(entity)

        all_data_group_result.append({
            "text": labeled_origin_text,
            "group_result": group_result
        })
    
    return all_data_group_result

def _recombine_grouped_label_data(all_data_group_result:Dict[str, List[str]]) -> Dict[str, List[str]]:
    for item in all_data_group_result:
        group_result = item["group_result"]
        group_result.pop("主要病变",[])
        group_result.pop("移除处理",[])
        for key in list(group_result.keys()):
            if key != "non_attr_entities":
                group_result["non_attr_entities"].extend(group_result.pop(key))
    return all_data_group_result

alpaca_data = [
  {
    "instruction": "",
    "input": "1患者仰卧位，全麻起效后常规消毒铺巾 2头偏左侧，开口器开口，口内消毒，拉钩牵拉口角，充分暴露右舌肿物，沿肿物外缘1厘米将肿物完整切除并取切缘，送冰冻，术中冰冻提示：“右舌”黏膜鳞状细胞癌。送检切缘：“前、后、上、下、底”均阴性（-）。 3冲洗创面后，充分止血，无明显出血点后，设计梭形瓣关闭创面，并于舌根处悬吊一牵引线以防止舌后坠。 手术结束，病人安返病房。",
    "output": "术后患者去向:安返病房\n移除处理:切除\n移除处理:处理对象及所在部位:右舌\n移除处理:处理对象（删除）:肿物\n主要病变:肿物\n主要病变:病变部位:右舌"
  },
  {
    "instruction": "",
    "input": "1、患者仰卧，头正中位，鼻插管全麻，导尿。常规消毒、铺巾，口内消毒。 2、设计左侧颊黏膜肿块周围1cm梭形切口，切开黏膜，完整切除肿块，送检冰冻。术中冰冻病理：“左颊”黏膜鳞状上皮乳头状增生及中-重度异常增生，局灶癌变。前、后、上、下、底”均阴性（-）。 3、彻底止血，冲洗，拔除左下智齿。缝合拔牙创牙龈。设计软组织邻近瓣修复创面。补片修整后覆盖创面，边缘缝合，碘仿打包加压包扎。 4、术毕，术顺，患者安返病房。",
    "output": "术后患者去向:安返病房\n移除处理:切除\n移除处理:处理对象及所在部位:左侧颊黏膜\n修复重建技术:设计邻近瓣\n修复重建技术:材料来源:左侧颊黏膜\n修复重建技术:修复重建部位:左侧颊黏膜"
  },
  {
    "instruction": "",
    "input": "1、患者仰卧，头朝右侧，全麻鼻插管，起效后常规消毒、铺巾、导尿、缝合1针固定气管插管。 2、先行原发灶切除，肿物位于下牙龈累及颊舌侧黏膜，沿肿物外周1cm处做手术切口，行龈颌联合根治，以动力系统边缘性切除累及下颌骨，在正常组织内将肿物完整切除，送快速冰冻病理检查，并取切缘送冰冻，术中冰冻病理示：“右下前牙区牙龈”黏膜鳞状细胞癌；送检切缘：“左后”黏膜鳞状上皮瘤样增生及轻度异常增生；“前、左前、右前、后、左后、右后、底、颏N、舌骨肌”均阴性（-）。术区严密止血。 3、行左侧肩胛舌骨上淋巴清扫术，做颏下、颌下乳突切口，切开皮肤、皮下组织，颈阔肌，翻瓣。翻瓣范围前至对侧二腹肌前腹，上至下颌下缘上1cm，后至乳突，下至肩胛舌骨肌以上。由前向后先清扫颏下淋巴结，于下颌骨下缘保护面神经下颌缘支，结扎面动脉远心端及面静脉，将颌下腺向后牵拉，拉钩牵拉下颌舌骨肌向前，结扎下颌下神经节，清扫颌下三角，结扎保护面动脉近心端以备吻合，于颈鞘前缘分离保护甲状腺上动脉以备吻合，打开颈鞘，保护颈动脉及颈静脉迷走神经，清扫肩胛舌骨肌以上淋巴结，牵拉胸锁乳突将上端及二腹肌后腹，分离并保护副神经，清扫IIb区淋巴结。 4、行右侧根治性颈淋巴清扫术，沿设计的切口逐层切开左颈部皮肤、皮下层、颈阔肌层，沿颈阔肌深面向后翻瓣至斜方肌前缘，向上翻瓣至下颌骨下缘，向下达锁骨上缘，向前达颈中线，切断结扎胸锁乳突肌，沿锁骨上分离肩胛舌骨肌，打开颈鞘，肿块与颈内静脉及颈外动脉界限欠清，结扎颈内静脉及颈外动脉，沿椎前筋膜浅面斜方肌前缘向上行颈部淋巴结、脂肪结蒂组织等的清扫至颅底，分离颈内动脉，切断胸锁乳突肌，暴露对侧二腹肌，清扫颌下淋巴组织，结扎切断舌神经襻和颌下腺导管，切除颌下腺。结扎、切断面后静脉及颌外动脉。术中见肿块与腮腺下极黏连，切除部分腮腺下极及咬肌，术中将左侧副神经一并切除，至此，完全清扫颈部淋巴组织。转邻近组织瓣修复口内手术缺损，冲洗止血，颈部及腿部放置负压引流管。 5、手术结束，术中出血约300ml，未输血，术后予抗炎、支持及营养血管治疗，术后入ICU观察患者生命体征、皮瓣色形质情况及引流量。",
    "output": "手术名称:原发灶切除\n主要病变:肿物\n主要病变:病变部位:下牙龈\n主要病变:病变影响对象:颊舌侧黏膜\n主要病变:病变影响结果:累及\n手术名称:龈颌联合根治\n移除处理:切除\n移除处理:处理对象（删除）:下颌骨\n移除处理:切除\n移除处理:处理对象（删除）:肿物\n手术名称:左侧肩胛舌骨上淋巴清扫术\n移除处理:清扫\n移除处理:处理对象及所在部位:颏下淋巴结\n移除处理:清扫\n移除处理:处理对象及所在部位:颌下三角\n移除处理:清扫\n移除处理:处理对象及所在部位:肩胛舌骨肌以上淋巴结\n移除处理:清扫\n移除处理:处理对象及所在部位:IIb区淋巴结\n手术名称:右侧根治性颈淋巴清扫术\n移除处理:清扫\n移除处理:处理对象及所在部位:颈部淋巴结、脂肪结蒂组织\n移除处理:清扫\n移除处理:处理对象及所在部位:颌下淋巴组织\n移除处理:切除\n移除处理:处理对象（删除）:颌下腺\n移除处理:切除\n移除处理:处理对象及所在部位:部分腮腺下极及咬肌\n移除处理:切除\n移除处理:处理对象及所在部位:左侧副神经\n移除处理:清扫\n移除处理:处理对象及所在部位:颈部淋巴组织\n术后患者去向:入ICU\n出血标志:出血\n出血标志:出血量:300ml\n修复重建技术:转瓣\n修复重建技术:材料来源:邻近组织\n修复重建技术:修复重建部位:口内"
  }
]

json_data = match_grouped_label_data(LABEL_LIST_DICT,alpaca_data)

# print(json_data)
print(_recombine_grouped_label_data(json_data))


[{'text': '1患者仰卧位，全麻起效后常规消毒铺巾 2头偏左侧，开口器开口，口内消毒，拉钩牵拉口角，充分暴露右舌肿物，沿肿物外缘1厘米将肿物完整切除并取切缘，送冰冻，术中冰冻提示：“右舌”黏膜鳞状细胞癌。送检切缘：“前、后、上、下、底”均阴性（-）。 3冲洗创面后，充分止血，无明显出血点后，设计梭形瓣关闭创面，并于舌根处悬吊一牵引线以防止舌后坠。 手术结束，病人安返病房。', 'group_result': {'non_attr_entities': ['术后患者去向:安返病房']}}, {'text': '1、患者仰卧，头正中位，鼻插管全麻，导尿。常规消毒、铺巾，口内消毒。 2、设计左侧颊黏膜肿块周围1cm梭形切口，切开黏膜，完整切除肿块，送检冰冻。术中冰冻病理：“左颊”黏膜鳞状上皮乳头状增生及中-重度异常增生，局灶癌变。前、后、上、下、底”均阴性（-）。 3、彻底止血，冲洗，拔除左下智齿。缝合拔牙创牙龈。设计软组织邻近瓣修复创面。补片修整后覆盖创面，边缘缝合，碘仿打包加压包扎。 4、术毕，术顺，患者安返病房。', 'group_result': {'non_attr_entities': ['术后患者去向:安返病房', ['修复重建技术:设计邻近瓣', '修复重建技术:材料来源:左侧颊黏膜', '修复重建技术:修复重建部位:左侧颊黏膜']]}}, {'text': '1、患者仰卧，头朝右侧，全麻鼻插管，起效后常规消毒、铺巾、导尿、缝合1针固定气管插管。 2、先行原发灶切除，肿物位于下牙龈累及颊舌侧黏膜，沿肿物外周1cm处做手术切口，行龈颌联合根治，以动力系统边缘性切除累及下颌骨，在正常组织内将肿物完整切除，送快速冰冻病理检查，并取切缘送冰冻，术中冰冻病理示：“右下前牙区牙龈”黏膜鳞状细胞癌；送检切缘：“左后”黏膜鳞状上皮瘤样增生及轻度异常增生；“前、左前、右前、后、左后、右后、底、颏N、舌骨肌”均阴性（-）。术区严密止血。 3、行左侧肩胛舌骨上淋巴清扫术，做颏下、颌下乳突切口，切开皮肤、皮下组织，颈阔肌，翻瓣。翻瓣范围前至对侧二腹肌前腹，上至下颌下缘上1cm，后至乳突，下至肩胛舌骨肌以上。由前向后先清扫颏下淋巴结，于下颌骨下缘保护面神经下颌缘支，结扎面动脉远心端及面静脉，将颌下腺向后牵拉，拉钩牵拉下颌舌骨肌向前，结扎下颌下神经节，清扫颌下三角，结扎保护面

In [6]:
a = [1,2,3,4,5]
print(a[1:3])

[2, 3]
